In [1]:
import numpy as np
import pandas as pd
import jieba
import string
# import fasttext
import platform
platform.system()
# import tensorflow as tf
# sess = tf.InteractiveSession()

C:\users\usz0b3l\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


'Windows'

In [2]:
cwe = np.load("zh.bin.syn0.npy")

In [4]:
cwe.shape

(50101, 300)

In [5]:
cwe_neg = np.load("zh.bin.syn1neg.npy")

In [6]:
cwe_neg.shape

(50101, 300)

# Word Embedding Matrix Lookup (cross platform)
Reference: [https://github.com/facebookresearch/fastText/tree/master/python](https://github.com/facebookresearch/fastText/tree/master/python)

In [2]:
if platform.system() == 'Windows':
    with open("zh/wiki.zh.vec", "r", encoding='utf-8') as fin:
        words = fin.readlines()
elif platform.system() == 'Darwin':  # Mac OSX
    with open("zh/wiki.zh.vec", "r") as fin:
        words = fin.readlines()

In [3]:
len(words)

332648

In [4]:
if platform.system() == 'Windows':
    wiki_vocab = [x.split(' ')[0] for x in words[1:]]
elif platform.system() == 'Darwin':
    wiki_vocab = [x.split(' ')[0].decode('utf-8') for x in words[1:]]

In [5]:
len(wiki_vocab)

332647

In [6]:
print(wiki_vocab[100])

日本


In [ ]:
# wiki_embeddings = np.array([x.strip().split(' ')[1:] for x in words[1:]])  # will probably run out of memory

In [7]:
def read_vocab(file_path):
    if platform.system() == 'Windows':
        with open(file_path, 'r', encoding='utf-8') as fin:
            vocab = [x.strip() for x in fin.readlines()]
            vocab = [x for x in vocab if x != '']
    elif platform.system() == 'Darwin':
        with open(file_path, 'r') as fin:
            vocab = [x.strip() for x in fin.readlines()]
            vocab = [x.decode('utf-8') for x in vocab if x != '']
    return vocab

In [8]:
vocab = read_vocab('../01_WebScraping/vocab.txt')

In [9]:
len(vocab)

9827

In [10]:
print(vocab[0])

沉静


In [4]:
def custom_tokenizer(text):
    return [token for token in jieba.cut(text) if token not in string.punctuation+" " and token != '\n']

In [2]:
def read_lyrics(file_path):
    if platform.system() == 'Windows':
        lyrics = [line.strip() for line in open(file_path, 'r', encoding='utf-8').readlines()]
        lyrics = [custom_tokenizer(text) for text in lyrics]
    elif platform.system() == 'Darwin':
        lyrics = [line.strip() for line in open(file_path, 'r').readlines()]
        lyrics = [custom_tokenizer(text) for text in lyrics]
    return lyrics

In [5]:
lyrics = read_lyrics('../01_WebScraping/lyrics.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\users\usz0b3l\AppData\Local\Temp\jieba.cache
Loading model cost 0.937 seconds.
Prefix dict has been built succesfully.


In [6]:
for item in lyrics[:3]:
    print(len(item))

82
210
161


# Token look up

In [15]:
def token_lookup(token, vocab, word_embedding):
    try:
        return [float(x) for x in word_embedding[vocab.index(token)+1].strip().split(' ')[1:]]
    except ValueError:
        print('Token not in vocabulary!')
        # See https://github.com/facebookresearch/fastText#obtaining-word-vectors-for-out-of-vocabulary-words for more details
        return ''

In [16]:
len(token_lookup(vocab[5], wiki_vocab, words))

300

In [18]:
# Skipgram model
model = fasttext.load_model('zh/wiki.zh.bin')
print(model['king'])  # get the vector of the word 'king'

Exception: fastText: Cannot load zh/wiki.zh.bin due to C++ extension failed to allocate the memory